In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import regex as re
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [2]:
train_df = pd.read_csv("train.csv") # Load Train
test_df = pd.read_csv("test.csv") # Load test

In [3]:
train_df.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
train_df.isna().sum()


id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
test_df.isna().sum()


id          0
title     122
author    503
text        7
dtype: int64

In [6]:
train_df = train_df.fillna(' ') # Fill missing values
test_df = test_df.fillna(' ') 
print(train_df.isna().sum())
print(test_df.isna().sum())

id        0
title     0
author    0
text      0
label     0
dtype: int64
id        0
title     0
author    0
text      0
dtype: int64


In [7]:
def apply_stemming(content, stemmer):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [8]:
train_df['content'] =  train_df['author'] + ' ' + train_df['title'] 

In [9]:
train_df.content

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [10]:
stemmer = PorterStemmer()
train_df['content'] = train_df['content'].apply(lambda content: apply_stemming(content, stemmer))

In [11]:
train_df.columns

Index(['id', 'title', 'author', 'text', 'label', 'content'], dtype='object')

In [12]:
X = train_df['content'].values
y = train_df.label.values
X

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [13]:
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(X)
X_vec

<20800x17128 sparse matrix of type '<class 'numpy.int64'>'
	with 210687 stored elements in Compressed Sparse Row format>

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, stratify=y, random_state=42)

In [15]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.9860576923076924

In [16]:
lr = LogisticRegression()
lr_params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100], # Regularization parameter
    'penalty': ['l1', 'l2'], # Regularization type
}
gs = GridSearchCV(lr, lr_params)
gs.fit(X_train, y_train)
gs.score(X_test, y_test)

d:\Software\Python 9.1.2\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Software\Python 9.1.2\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Software\Python 9.1.2\lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "d:\Software\Python 9.1.2\lib\site-packages\sklearn\linear_model\_logistic.py", line 1193, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)

0.9903846153846154

In [17]:
SVM = LinearSVC()
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],  # Usually 'l2' for LinearSVC, 'l1' can be used for a sparse model but is not supported with the default solver
    'max_iter': [1000, 5000, 10000],
    'dual': [False]  # dual=False is faster when n_samples > n_features
}
gs_svm = GridSearchCV(SVM, param_grid)
gs_svm.fit(X_train, y_train)
gs_svm.score(X_test, y_test)

0.9925480769230769

In [18]:
gs_svm.best_params_

{'C': 1, 'dual': False, 'max_iter': 1000, 'penalty': 'l2'}

In [19]:
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [20]:
test_df['content'] = test_df['title'] + ' ' + test_df['author']
test_df['content'] = test_df['content'].apply(lambda content: apply_stemming(content, stemmer))
X_test = test_df.content.values
X_test_vec = vectorizer.transform(X_test)

In [21]:
svm = gs_svm.best_estimator_
y_pred = svm.predict(X_test_vec)
submission_data = {
    'id': test_df['id'],
    'label': y_pred
}
submission = pd.DataFrame(submission_data)
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


In [22]:
submission.to_csv("submission_data.csv", index=False)